In [30]:
#-----------------------------------------
# importamos los paquetes que vamos a usar
#-----------------------------------------

import os
import shutil
import numpy as np
import glob                         # para hacer listas de archivo
import subprocess                   # para usar subprocesos

In [31]:
ant = 'A1'                                         # antena usada (A1 o A2)    
obs_template = np.loadtxt( './templates/obs_template_' + ant + '.dat' , skiprows=0 , dtype=str , usecols=(0) )  # observación originalmente usadada como template

In [32]:
#-----------------------------------------------
# generamos carpetas para guardar los resultados
#-----------------------------------------------

dirName = "./results_nbins_" + ant            # nombre de la carpeta en que guardaremos los resultados
if not os.path.exists(dirName):
    os.mkdir(dirName)

    
os.chdir(dirName + '/')                       # entramos en la carpeta en la cual guadaremos los resultados
    
if not os.path.exists('./timing_files'):
    os.mkdir('./timing_files')                # creamos la carpeta que contendrá los archivos .tim       
    
os.chdir('..')                                # salimos de la carpeta en la cual guadaremos los resultados
    
    
shutil.copy('./J0437-4715.par', './results_nbins_' + ant + '/timing_files/')     # copiamos el archivo .par a la nueva carpeta para poder usar tempo2

'./results_nbins_A1/timing_files/J0437-4715.par'

In [34]:
#--------------------------------------------------------
# Ahora trabajamos con los .pfds de nbin=1024,512,256,...
#--------------------------------------------------------

for nbin in (32, 64, 128, 256, 512, 1024):

    folder = './' + ant + '_' + str(nbin) + '/'                # nombre de la carpeta que contiene los .pfds
    
    template = 'template_' + str(nbin) + '_' + ant + '.std.sm' # nombre del template
    
    shutil.copy('./templates/' + template, folder)             # copiamos el template a la carpeta

    
    os.chdir(folder)                            # entramos al directorio que contiene los .pfds
    pfd_files = glob.glob('*pfd')               # leemos los .pfds
    

#-----------------------------------------
# generamos los TOAs (el archivo .tim)
#-----------------------------------------

# primero, generamos un string que contenga todos los archivos .pfd menos el que usamos como template

    pfd_files.remove(str(obs_template))     # sacamos de la lista de observaciones a la que usamos como templates

    archivos = ''
    for pfd in pfd_files:
        archivos = archivos + pfd + ' '     # hacemos un string que contenga al resto de las observaciones

# ahora llamamos a pat para generar el archivo .tim

    timing_file = "timing_" + str(nbin) + ".tim"       # nombre del archivo .tim

    subprocess.check_call( "pat -FT -A PGS -s " + template + " -f >" + timing_file + " 'tempo2' " + archivos, shell = True)
    
    os.chdir('..')                                     # salimos del directorio que contiene los .pfds
    
    shutil.copy(folder + '/' + timing_file, "./results_nbins_" + ant + '/timing_files/')             # copiamos el template a la carpeta

In [29]:
#-----------------------------------------
# generamos los TOAs (el archivo .tim)
#-----------------------------------------

    

# primero, generamos un string que contenga todos los archivos .pfd menos el que usamos como template
    
    pfd_files.remove(obs_template)  
        
    archivos = ''
    for pfd in pfd_files:
        archivos = archivos + pfd + ' '

# ahora llamamos a pat para generar el archivo .tim

    timing_file = timing_file = "./results_nbins_" + ant + "/timing_files/timing_" + str(nbin) + ".tim"       # nombre del archivo .tim
    
    template = 

    subprocess.check_call( "pat -FT -A PGS -s " + template + " -f >" + timing_file + " 'tempo2' " + archivos, cwd=timing_folder, shell = True)


In [ ]:




#-----------------------------------------
# calculamos el rms usando tempo2
#-----------------------------------------

    residue_file = "residues_" + str(nbin) + "_" + str(ant) + ".dat"   # nombre del archivo de los residuos

    subprocess.check_call( "tempo2 >" + residue_file + " -us -f J0437-4715.par " + timing_file, cwd=timing_folder, shell=True)

# abrimos el archivo generado por tempo2, donde se encuentra el RMS que buscamos

    RMS = np.genfromtxt ( str(ant) + '_' + str(nbin) + '/' + residue_file, comments="none", dtype=float, skip_header=18, max_rows=1, usecols=(10) )

    print(str(nbin) + ' ' + str(RMS) )

In [2]:
#----------------------------------------------------------------------------
# Partimos de un conjunto de observaciones mezcladas con nbin=1024 y nbin=512
# y nos quedamos con los .pfds de nbin=1024
#----------------------------------------------------------------------------

# creamos una lista con todos los archivos .pfd

ant = 'A2'                                                # antena usada (A1 o A2)

timing_folder = './' + str(ant) + '/'
pfd_files_total = glob.glob(timing_folder+'*pfd')         # lista de archivos .pfd
output = "output.dat"                                     # nombre del archivo de salida del cual vamos leyendo los headers

# para cada archvio .pfd, generamos un archivo con su header y leemos su nbins y su nchan

n=0
m=0
pfd_files_1024 = []
pfds_1024_128 = []

for pfd in pfd_files_total:
    
    subprocess.check_call( "psrstat >" + output + " " + pfd , shell=True )
    
    n_bins = np.genfromtxt ( output, comments="none", dtype=int, skip_header=1, max_rows=1, usecols=(6) )
    n_chan = np.genfromtxt ( output, comments="none", dtype=int, skip_header=2, max_rows=1, usecols=(5) )

    
    if n_bins==1024:                                     # si el .pfd tiene nbin=1024
        pfd_files_1024.append(pfd)                            # entonces lo guardamos en la lista
        n += 1                                           # y aumentamos en 1 el numero de pfds con nbin=1024
        
        if n_chan==128:                                  # si el .pfd tiene nchan=128           
            pfds_1024_128.append(pfd)                    # entonces lo guardamos en la lista con nchan=128
            m += 1                              

In [29]:
#------------------------------------------------------------------------
# Esto es para mover las observaciones con nbins=1024 a una nueva carpeta
#------------------------------------------------------------------------

ant = 'A2'                                                # antena usada (A1 o A2)  
timing_folder_1024 = './' + str(ant) + '_1024/'

for i in range(len(pfd_files_1024)):
    shutil.copy(pfd_files_1024[i], timing_folder_1024)
    shutil.copy(pfd_files_1024[i] + ".polycos", timing_folder_1024)
    shutil.copy(pfd_files_1024[i] + ".bestprof", timing_folder_1024)

In [80]:

#--------------------------------------------------------
# Ahora trabajamos con los .pfds de nbin=1024,512,256,...
#--------------------------------------------------------

for nbin in (32, 64, 128, 256, 512, 1024):

    ant = 'A2'                                                # antena usada (A1 o A2)

    timing_folder = './' + str(ant) + '_' + str(nbin) + '/'   # nombre de la carpeta que contiene los .pfds

    os.chdir(timing_folder)                     # entramos al directorio que contiene los .pfds
    pfd_files = glob.glob('*pfd')               # leemos los .pfds
    os.chdir('..')                              # salimos del directorio que contiene los .pfds

#    print(len(pfd_files))



#-----------------------------------------
# generamos los TOAs (el archivo .tim)
#-----------------------------------------

    template = "template_" + str(nbin) + "_" + str(ant) + ".std.sm"       # nombre del template
    timing_file = "./timing_" + str(nbin) + "_" + str(ant) + ".tim"       # nombre del archivo .tim

#    print("template = " + str(template) )
#    print("timing file = " + str(timing_file) )

# primero, generamos un string que contenga todos los archivos .pfd
# menos el que usamos como template

    pfd_files.remove('prepfold_20191204_230627_PSR_0437-4715.pfd')

    archivos = ''
    for pfd in pfd_files:
        archivos = archivos + pfd + ' '
    
#print(archivos)
       
# ahora llamamos a pat para generar el archivo .tim

    subprocess.check_call( "pat -FT -A PGS -s " + template + " -f >" + timing_file + " 'tempo2' " + archivos, cwd=timing_folder, shell = True)




#-----------------------------------------
# calculamos el rms usando tempo2
#-----------------------------------------

    residue_file = "residues_" + str(nbin) + "_" + str(ant) + ".dat"   # nombre del archivo de los residuos

    subprocess.check_call( "tempo2 >" + residue_file + " -us -f J0437-4715.par " + timing_file, cwd=timing_folder, shell=True)

# abrimos el archivo generado por tempo2, donde se encuentra el RMS que buscamos

    RMS = np.genfromtxt ( str(ant) + '_' + str(nbin) + '/' + residue_file, comments="none", dtype=float, skip_header=18, max_rows=1, usecols=(10) )

    print(str(nbin) + ' ' + str(RMS) )

32 38.313
64 1.201
128 0.982
256 0.849
512 0.838
1024 0.838


In [76]:
#--------------------------------------------------------
# Ahora trabajamos con los .pfds de nbin=1024,512,256,...
#--------------------------------------------------------

ant = 'A2'                                                # antena usada (A1 o A2)
nbin = 1024                                                 # numero de bines en fase

timing_folder = './' + str(ant) + '_' + str(nbin) + '/'

os.chdir(timing_folder)                     # entramos al directorio que contiene los .pfds
pfd_files = glob.glob('*pfd')               # leemos los .pfds
os.chdir('..')                              # salimos del directorio que contiene los .pfds

#print(pfd_files)

In [77]:
#-----------------------------------------
# generamos los TOAs (el archivo .tim)
#-----------------------------------------

template = "template_" + str(nbin) + "_" + str(ant) + ".std.sm"  # nombre del template
timing_file = "./timing_" + str(nbin) + "_" + str(ant) + ".tim"       # nombre del archivo .tim

print("template = " + str(template) )
print("timing file = " + str(timing_file) )

# primero, generamos un string que contenga todos los archivos .pfd
# menos el que usamos como template

pfd_files.remove('prepfold_20191204_230627_PSR_0437-4715.pfd')

archivos = ''
for pfd in pfd_files:
    archivos = archivos + pfd + ' '
    
#print(archivos)
       
# ahora llamamos a pat para generar el archivo .tim

subprocess.check_call( "pat -FT -A PGS -s " + template + " -f >" + timing_file + " 'tempo2' " + archivos, cwd=timing_folder, shell = True)

template = template_1024_A2.std.sm
timing file = ./timing_1024_A2.tim


0

In [48]:
#--------------------------------------------
# En caso de trabajar con observaciones de A1
#--------------------------------------------

subprocess.check_call( "sed -i -e 's/1414.8750/1414.4375/g' " + timing_file , shell=True )

0

In [78]:
#-----------------------------------------
# calculamos el rms usando tempo2
#-----------------------------------------

residue_file = "residues_" + str(nbin) + "_" + str(ant) + ".dat"   # nombre del archivo de los residuos

subprocess.check_call( "tempo2 >" + residue_file + " -us -f J0437-4715.par " + timing_file, cwd=timing_folder, shell=True)

# abrimos el archivo generado por tempo2, donde se encuentra el RMS que buscamos

RMS = np.genfromtxt ( str(ant) + '_' + str(nbin) + '/' + residue_file, comments="none", dtype=float, skip_header=18, max_rows=1, usecols=(10) )

print("RMS = " + str(RMS) )

RMS = 0.838
